# INF8225 - PIAD - Anomaly Detection







## Import & Git

In [ ]:
# Import git projet

!git clone https://github.com/deca1111/Anomaly_Detection_Comparison.git

# changement de folder
%cd /content/Anomaly_Detection_Comparison/

Cloning into 'INF8225-Projet'...
remote: Enumerating objects: 2104, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 2104 (delta 18), reused 61 (delta 16), pack-reused 2041
Receiving objects: 100% (2104/2104), 469.55 MiB | 41.59 MiB/s, done.
Resolving deltas: 100% (676/676), done.
/content/INF8225-Projet


In [ ]:
# Instalation modules utiles
# ! pip install -r requirements.txt

! pip install tensorboardX
! pip install pyyaml
! pip install patool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
# Import utiles

from anomaly_detection.piad.train import Trainer
from anomaly_detection.piad.evaluate import evaluate

import torch
import torch.nn as nn

import numpy as np
import yaml

import time
import os
import sys

import urllib.request
import patoolib

import subprocess
from multiprocessing import Process

import pandas as pd

import argparse

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cuda


# Import Dataset

## Unzip Dataset

In [ ]:
compressedFile = "/content/Anomaly_Detection_Comparison/data/data/nih/subset_300_zip/images_subset_300.zip"

targetImagesRoot = "/content/Anomaly_Detection_Comparison/data/data/nih_300/"

# Création folder
os.makedirs(targetImagesRoot, exist_ok=True)

# Unzip
patoolib.extract_archive(compressedFile, outdir=targetImagesRoot)

INFO patool: Extracting /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip ...
INFO:patool:Extracting /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip ...
INFO patool: running /usr/bin/7z x -o/content/INF8225-Projet/data/data/nih_300/ -- /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip
INFO:patool:running /usr/bin/7z x -o/content/INF8225-Projet/data/data/nih_300/ -- /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip extracted to `/content/INF8225-Projet/data/data/nih_300/'.
INFO:patool:... /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip extracted to `/content/INF8225-Projet/data/data/nih_300/'.


'/content/INF8225-Projet/data/data/nih_300/'

## Vérification du Subset

In [ ]:
listImagesFile = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300.txt"

rootImages = "/content/Anomaly_Detection_Comparison/data/data/nih_300/"

# vérification que l'intégralité des images du subset est présent

with open(listImagesFile, 'r') as f:
    listImages = f.read().splitlines()

imagesManquantes = []
nbImages = 0
for image in listImages:
    if not os.path.exists(rootImages + image):
        imagesManquantes.append(image)
    else :
        nbImages += 1

if len(imagesManquantes) > 0:
  print(f"Images manquantes : {imagesManquantes}")
else:
  print(f"Tous les images du subset sont présentes [{nbImages}/{len(listImages)}]")

Tous les images du subset sont présentes [7311/7311]


# Entrainement et évaluation

In [ ]:
# Config piad
model = "piad"
dataset = "nih"
exp = "with_cv"
_class = "subset"


RUNS = ['0', '1', '2']

configs = []

for run in RUNS:

  config = f"configs/{model}/{dataset}/final/{exp}/class_{_class}/run_{run}/train_eval.yaml"

  print(config)
  configs.append(config)

def _load_config(path):
    with open(path, 'r') as stream:
        config = yaml.load(stream, Loader=yaml.FullLoader)
    return config

configs/piad/nih/final/with_cv/class_subset/run_0/train_eval.yaml
configs/piad/nih/final/with_cv/class_subset/run_1/train_eval.yaml
configs/piad/nih/final/with_cv/class_subset/run_2/train_eval.yaml


## Run 0

### Training

In [ ]:
start_time = time.time()
trainer = Trainer(_load_config(configs[0]))
trainer.train()
print(f"Training is complete. Took: {(time.time() - start_time) / 60:.02f}m")
del trainer

adam_kwargs:
  ddis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  edis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  enc_dec:
    betas:
    - 0.5
    - 0.99
    lr: 0.0002
batch_size: 32
checkpoint_root: ./data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0
ddis:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 128
    - 128
  type: residual9
dec:
  kwargs:
    inner_dims:
    - 256
    - 256
    - 128
    - 64
    - 32
  type: residual9
edis:
  kwargs:
    inner_dims:
    - 1024
    - 1024
enc:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 256
    - 256
  type: residual9
finetune_from: null
image_adv_loss:
  loss_kwargs:
    gradient_penalty: 10
    lambd: 1
    norm_penalty: 0.001
  loss_type: wasserstein
image_dim: 1
image_rec_loss:
  loss_kwargs:
    feature_weights:
      r42: 1
    img_weight: 0
  loss_type: relative_perceptual_L1
image_res: 64
image_sample_iter: 5000
initial_image_res: 64
iters: 40000
latent_adv_loss:
  loss_kwargs:
  

  0%|          | 0/40000 [00:00<?, ?it/s]

================================== MODELS ON LARGEST RESOLUTION ==================================
====================== Encoder =============================
STABNetwork(
  (model): Sequential(
    (preprocess_res_64): Sequential(
      (0): ConvBlock(
        (model): Sequential(
          (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (1): PreActResnetBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ConvBlock(
            (model): Sequential(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): LeakyReLU(negative_slope=0.2)
            )
          )
          (2): ConvBlock(
            (model): Sequential(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
          )
        )
        (skipcon): Sequential()
      )
    )
    (res_64): Sequential(
      (0): PreActResnetBlockDown(
        (

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth

  0%|          | 0.00/548M [00:00<?, ?B/s]
  2%|▏         | 10.0M/548M [00:00<00:05, 105MB/s]
  5%|▍         | 27.1M/548M [00:00<00:03, 148MB/s]
  8%|▊         | 44.9M/548M [00:00<00:03, 166MB/s]
 11%|█▏        | 63.0M/548M [00:00<00:02, 175MB/s]
 15%|█▍ 

Starting model training ...


100%|██████████| 40000/40000 [2:05:40<00:00,  5.46it/s]

Model training is complete.
Training is complete. Took: 125.75m


### Sauvegarde des poids du modèle dans un zip & Copie dans le drive

In [ ]:
targetFolder = f"/content/Anomaly_Detection_Comparison/data/checkpoint_tmp/{model}/{dataset}/final/{exp}/class_{_class}/run_{RUNS[0]}"
os.makedirs(targetFolder, exist_ok=True)

modelName = f"{model}__{dataset}__{exp}__class_{_class}__run_{RUNS[0]}"
outputFile = f"/content/Anomaly_Detection_Comparison/data/model_{modelName}.zip"

# Sauvegarde dans un fichier zip

!zip -r $outputFile $targetFolder

updating: content/INF8225-Projet/data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0/ (stored 0%)
updating: content/INF8225-Projet/data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0/checkpoint.tar (deflated 7%)
updating: content/INF8225-Projet/data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0/anomaly_detection_niter_40000.tar (deflated 7%)
updating: content/INF8225-Projet/data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0/anomaly_detection.tar (deflated 7%)


In [ ]:
# Ajout du drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
inputFile = outputFile
outputFiles = f"/content/drive/MyDrive/Polytechnique/INF8225/Projet/modelSave/{modelName}.zip"

!cp $inputFile $outputFiles

### Sauvegarde des logs de l'entrainement dans un zip & Copie dans le drive

In [ ]:
# Sauvegarde des logs dans un zip puis dans le drive

targetLogsFolder = f"/content/Anomaly_Detection_Comparison/data/logs_tmp/{model}/{dataset}/final/{exp}/class_{_class}/run_{RUNS[0]}"
os.makedirs(targetLogsFolder, exist_ok=True)

logsFile = f"/content/Anomaly_Detection_Comparison/data/logs_{modelName}.zip"

!zip -r $logsFile $targetLogsFolder

driveOutputRoot = "/content/drive/MyDrive/Polytechnique/INF8225/Projet/logs/"
os.makedirs(driveOutputRoot, exist_ok=True)

outputFiles = os.path.join(driveOutputRoot, f"logs_{modelName}.zip")

!cp $logsFile $outputFiles


updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/ (stored 0%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/tensorboard/ (stored 0%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/tensorboard/events.out.tfevents.1713037499.fdb8694e1136 (stored 0%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/tensorboard/train/ (stored 0%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/tensorboard/train/ddis/ (stored 0%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/tensorboard/train/ddis/norm/ (stored 0%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_subset/run_0/tensorboard/train/ddis/norm/events.out.tfevents.1713037821.fdb8694e1136 (deflated 66%)
updating: content/INF8225-Projet/data/logs_tmp/piad/nih/final/with_cv/class_

### Évaluation

In [ ]:
start_time = time.time()

evaluate(_load_config(configs[0]))

print(f"Evaluation is complete. Took: {(time.time() - start_time) / 60:.02f}m")

adam_kwargs:
  ddis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  edis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  enc_dec:
    betas:
    - 0.5
    - 0.99
    lr: 0.0002
batch_size: 32
checkpoint_root: ./data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0
ddis:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 128
    - 128
  type: residual9
dec:
  kwargs:
    inner_dims:
    - 256
    - 256
    - 128
    - 64
    - 32
  type: residual9
edis:
  kwargs:
    inner_dims:
    - 1024
    - 1024
enc:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 256
    - 256
  type: residual9
finetune_from: null
image_adv_loss:
  loss_kwargs:
    gradient_penalty: 10
    lambd: 1
    norm_penalty: 0.001
  loss_type: wasserstein
image_dim: 1
image_rec_loss:
  loss_kwargs:
    feature_weights:
      r42: 1
    img_weight: 0
  loss_type: relative_perceptual_L1
image_res: 64
image_sample_iter: 5000
initial_image_res: 64
iters: 40000
latent_adv_loss:
  loss_kwargs:
  

100%|██████████| 21/21 [00:00<00:00, 26.48it/s]

Model evaluation is complete. Results: 
   niter   ROC AUC
0  40000  0.884784
Evaluation is complete. Took: 0.06m


## All runs together (subprocess)

⚠ Je n'ai pas réussi à faire fonctionner cette partie dans colab, on ne fait donc qu'une run à la fois

In [ ]:
# def run_training(config):

#     start_time = time.time()
#     trainer = Trainer(_load_config(config))
#     trainer.train()
#     print(f"Training is complete. Took: {(time.time() - start_time) / 60:.02f}m")
#     del trainer

#     start_time = time.time()
#     evaluate(_load_config(config))
#     print(f"Evaluation is complete. Took: {(time.time() - start_time) / 60:.02f}m")

In [ ]:
# Lancement des 3 runs en mêmes temps
# for config in configs:
#     p = Process(target=run_training, args=(config,))
#     p.start()


adam_kwargs:
  ddis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  edis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  enc_dec:
    betas:
    - 0.5
    - 0.99
    lr: 0.0002
batch_size: 32
checkpoint_root: ./data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_0
ddis:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 128
    - 128
  type: residual9
dec:
  kwargs:
    inner_dims:
    - 256
    - 256
    - 128
    - 64
    - 32
  type: residual9
edis:
  kwargs:
    inner_dims:
    - 1024
    - 1024
enc:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 256
    - 256
  type: residual9
finetune_from: null
image_adv_loss:
  loss_kwargs:
    gradient_penalty: 10
    lambd: 1
    norm_penalty: 0.001
  loss_type: wasserstein
image_dim: 1
image_rec_loss:
  loss_kwargs:
    feature_weights:
      r42: 1
    img_weight: 0
  loss_type: relative_perceptual_L1
image_res: 64
image_sample_iter: 5000
initial_image_res: 64
iters: 40000
latent_adv_loss:
  loss_kwargs:
  

  0%|          | 0/40000 [00:00<?, ?it/s]

adam_kwargs:
  ddis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  edis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  enc_dec:
    betas:
    - 0.5
    - 0.99
    lr: 0.0002
batch_size: 32
checkpoint_root: ./data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_1
ddis:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 128
    - 128
  type: residual9
dec:
  kwargs:
    inner_dims:
    - 256
    - 256
    - 128
    - 64
    - 32
  type: residual9
edis:
  kwargs:
    inner_dims:
    - 1024
    - 1024
enc:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 256
    - 256
  type: residual9
finetune_from: null
image_adv_loss:
  loss_kwargs:
    gradient_penalty: 10
    lambd: 1
    norm_penalty: 0.001
  loss_type: wasserstein
image_dim: 1
image_rec_loss:
  loss_kwargs:
    feature_weights:
      r42: 1
    img_weight: 0
  loss_type: relative_perceptual_L1
image_res: 64
image_sample_iter: 5000
initial_image_res: 64
iters: 40000
latent_adv_loss:
  loss_kwargs:
  

  0%|          | 0/40000 [00:00<?, ?it/s]

adam_kwargs:
  ddis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  edis:
    betas:
    - 0.5
    - 0.99
    lr: 0.0005
  enc_dec:
    betas:
    - 0.5
    - 0.99
    lr: 0.0002
batch_size: 32
checkpoint_root: ./data/checkpoint_tmp/piad/nih/final/with_cv/class_subset/run_2
ddis:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 128
    - 128
  type: residual9
dec:
  kwargs:
    inner_dims:
    - 256
    - 256
    - 128
    - 64
    - 32
  type: residual9
edis:
  kwargs:
    inner_dims:
    - 1024
    - 1024
enc:
  kwargs:
    inner_dims:
    - 32
    - 64
    - 128
    - 256
    - 256
  type: residual9
finetune_from: null
image_adv_loss:
  loss_kwargs:
    gradient_penalty: 10
    lambd: 1
    norm_penalty: 0.001
  loss_type: wasserstein
image_dim: 1
image_rec_loss:
  loss_kwargs:
    feature_weights:
      r42: 1
    img_weight: 0
  loss_type: relative_perceptual_L1
image_res: 64
image_sample_iter: 5000
initial_image_res: 64
iters: 40000
latent_adv_loss:
  loss_kwargs:
  

  0%|          | 0/40000 [00:00<?, ?it/s]